In [1]:
import pandas as pd
import pathlib
import geopandas as gpd
import pypsa
import numpy as np
import shapely as spl

pd.options.display.max_columns = None
pd.options.display.max_rows = None

/opt/anaconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/networkclustering.py:24: UserWarning:

The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 



# Load the input data

In [2]:
base_path = pathlib.Path("/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/")
base_network_pypsa_earth = pypsa.Network(
    pathlib.Path(
        base_path,
        "geothermal-modelling",
        "workflow",
        "pypsa-earth",
        "networks",
        "US_2021",
        "base.nc",
    )
)
base_network_pypsa_usa = pd.read_csv(
    pathlib.Path(
        base_path,
        "pypsa-usa",
        "workflow",
        "resources",
        "Default",
        "usa",
        "lines_gis.csv",
    )
)
eia_base_network = gpd.read_file(
    pathlib.Path(
        base_path,
        "geothermal-modelling",
        "analysis",
        "gdrive_data",
        "data",
        "transmission_grid_data",
        "US_electric_transmission_lines_original.geojson",
    )
)
eia_base_network_modified = pd.read_csv(
    pathlib.Path(
        base_path,
        "geothermal-modelling",
        "analysis",
        "outputs",
        "modified_eia_base.csv",
    )
)
base_network_pypsa_earth_modified = pypsa.Network(
    pathlib.Path(
        base_path,
        "geothermal-modelling",
        "analysis",
        "outputs",
        "modified_pypsa_earth_base.nc",
    )
)
osm_lines_clean_modified = pd.read_csv(
    pathlib.Path(
        base_path,
        "geothermal-modelling",
        "analysis",
        "outputs",
        "modified_osm_lines_clean.csv",
    )
)
osm_lines_raw_modified = pd.read_csv(
    pathlib.Path(
        base_path,
        "geothermal-modelling",
        "analysis",
        "outputs",
        "modified_osm_lines_raw.csv",
    )
)
ipm_gdp_usa_level_1 = gpd.read_file(
    pathlib.Path(
        base_path,
        "geothermal-modelling",
        "analysis",
        "gdrive_data",
        "data",
        "shape_files",
        "ipm_v6_regions",
        "IPM_Regions_201770405.shp",
    )
)
ipm_region_capacities_earth = pd.read_csv(
    pathlib.Path(
        base_path,
        "geothermal-modelling",
        "analysis",
        "outputs",
        "pypsa_earth_ipm_capacities.csv",
    ),
    index_col=False,
)
ipm_region_capacities_usa = pd.read_csv(
    pathlib.Path(
        base_path,
        "geothermal-modelling",
        "analysis",
        "outputs",
        "pypsa_usa_ipm_capacities.csv",
    ),
    index_col=False,
)
ipm_region_capacities_usa_snom_above_5mw = pd.read_csv(
    pathlib.Path(
        base_path,
        "geothermal-modelling",
        "analysis",
        "pusa_snom_above_5mw",
        "outputs",
        "pypsa_usa_ipm_capacities.csv",
    ),
    index_col=False,
)

/opt/anaconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

/opt/anaconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

/opt/anaconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/components.py:318: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

/opt/anaconda3/envs/pypsa-earth/lib/python3.10/site-packages/pypsa/com

In [3]:
base_network_pypsa_usa[
    ["type", "v_nom", "s_nom", "lat1", "lon1", "lat2", "lon2"]
].drop_duplicates(keep="first").query("s_nom == 0.01").shape

(19304, 7)

In [14]:
base_network_pypsa_usa["lon_diff"] = (
    base_network_pypsa_usa["lon1"] - base_network_pypsa_usa["lon2"]
)
base_network_pypsa_usa["lat_diff"] = (
    base_network_pypsa_usa["lat1"] - base_network_pypsa_usa["lat2"]
)

In [17]:
lon_diff_array_snomother = base_network_pypsa_usa.query("s_nom != 0.01")[
    "lon_diff"
].unique()
lon_diff_array_snom001 = base_network_pypsa_usa.query("s_nom == 0.01")[
    "lon_diff"
].unique()
print(
    lon_diff_array_snomother.min(),
    np.median(lon_diff_array_snomother),
    lon_diff_array_snomother.mean(),
    lon_diff_array_snomother.max(),
)
print(
    lon_diff_array_snom001.min(),
    np.median(lon_diff_array_snom001),
    lon_diff_array_snom001.mean(),
    lon_diff_array_snom001.max(),
)
lat_diff_array_snomother = base_network_pypsa_usa.query("s_nom != 0.01")[
    "lat_diff"
].unique()
lat_diff_array_snom001 = base_network_pypsa_usa.query("s_nom == 0.01")[
    "lat_diff"
].unique()
print(
    lat_diff_array_snomother.min(),
    np.median(lat_diff_array_snomother),
    lat_diff_array_snomother.mean(),
    lat_diff_array_snomother.max(),
)
print(
    lat_diff_array_snom001.min(),
    np.median(lat_diff_array_snom001),
    lat_diff_array_snom001.mean(),
    lat_diff_array_snom001.max(),
)

-3.662000000000006 0.004699999999999704 0.00525376779026222 3.3519000000000005
-1.1980260339548465 0.045046882723497106 -0.0281806832737622 0.9457126305049997
-2.533200000000001 0.003299999999995862 0.0049549995863324104 2.923000000000002
-1.3546385338464901 -0.2998222536703672 -0.2735748539774196 0.5637706615019482


In [12]:
base_network_pypsa_usa.query("s_nom == 0.01")

array(['Al/St 240/40 2-bundle 220.0', 'Al/St 560/50 4-bundle 750.0',
       'Al/St 240/40 4-bundle 380.0', nan], dtype=object)

In [4]:
base_network_pypsa_usa.shape

(83973, 39)

In [4]:
np.sort(base_network_pypsa_usa.v_nom.unique())

array([ 69., 100., 115., 138., 161., 230., 345., 500., 765.])

In [11]:
base_network_pypsa_usa.type.unique()

array(['Al/St 240/40 2-bundle 220.0', 'Al/St 560/50 4-bundle 750.0', nan,
       'Al/St 240/40 4-bundle 380.0'], dtype=object)

In [13]:
base_network_pypsa_usa.head(1)

,Line,bus0,bus1,r,x,b,s_nom,v_nom,interconnect,type,carrier,underwater_fraction,g,s_nom_mod,s_nom_extendable,s_nom_min,s_nom_max,s_max_pu,capital_cost,build_year,lifetime,length,terrain_factor,num_parallel,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt,lat1,lon1,lat2,lon2,WKT_geometry
0,1,1,5,0.118496,0.567617,0.0,0.01,230.0,Eastern,Al/St 240/40 2-bundle 220.0,AC,0.0,0.0,0.0,False,0.0,inf,1.0,0.0,0,inf,0.0,1.0,1.0,-inf,inf,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.9761,-70.2211,43.9761,-70.2211,"LINESTRING (-70.2211 43.9761, -70.2211 43.9761)"


In [33]:
series_s_nom_pusa = base_network_pypsa_usa.s_nom
print(
    series_s_nom_pusa.min(),
    np.median(series_s_nom_pusa),
    series_s_nom_pusa.mean(),
    series_s_nom_pusa.max(),
)

0.01 136.07 212.61107748919298 4437.75


In [34]:
series_s_nom_pusa = base_network_pypsa_usa.query("s_nom >= 5.0").s_nom
print(
    series_s_nom_pusa.min(),
    np.median(series_s_nom_pusa),
    series_s_nom_pusa.mean(),
    series_s_nom_pusa.max(),
)

13.66 178.135 290.8851023201251 4437.75


In [35]:
series_s_nom_pe = base_network_pypsa_earth.lines.s_nom
print(
    series_s_nom_pe.min(),
    np.median(series_s_nom_pe),
    series_s_nom_pe.mean(),
    series_s_nom_pe.max(),
)

51.38994746056859 154.16984238170576 531.6432413162212 9250.190542902346


In [37]:
print(base_network_pypsa_usa.query("s_nom == 0.01").num_parallel.unique())

[1.]


In [38]:
19304 * 0.01

193.04

In [10]:
region_list = ["MIS_INKY", "MIS_LMI"]
ipm_region_capacities.query(
    "ipm_region_0.isin(@region_list) & ipm_region_1.isin(@region_list)"
).loc[:, ("ipm_region_0", "ipm_region_1", "IPM", "PyPSA", "factor_IPM_over_PyPSA")]

,ipm_region_0,ipm_region_1,IPM,PyPSA,factor_IPM_over_PyPSA
24,MIS_INKY,MIS_LMI,100.0,6166.793695,0.016216


In [11]:
ipm_region_capacities.query("factor_IPM_over_PyPSA > 1").head(2)

,Unnamed: 0,ipm_region_0,ipm_region_1,IPM,PyPSA,Error wrt IPM (%),Error wrt PyPSA (%),factor_IPM_over_PyPSA,coalesce
9,9,MIS_AR,MIS_LA,2688.0,128.474869,-95.220429,-1992.237983,20.922380,MIS_AR-->MIS_LA
11,11,MIS_AR,S_C_TVA,2546.0,2234.345542,-12.240945,-13.948355,1.139484,MIS_AR-->S_C_TVA


# HTML map for the IPM Regions

In [25]:
ipm_map = ipm_gdp_usa_level_1.explore(column="IPM_Region", cmap="jet")
ipm_map.save("interactive_map.html")

# Analysis on the transmission lines to spot missing lines

In [16]:
ipm_region_0_val = "WECC_NNV"  # "WECC_ID"
ipm_region_1_val = "WECC_PNW"  # "WECC_UT"
# state_0_val = "US-AR"
# state_1_val = "US-LA"
query_string_ipm = "(ipm_region_0 == @ipm_region_0_val & ipm_region_1 == @ipm_region_1_val) | (ipm_region_0 == @ipm_region_1_val & ipm_region_1 == @ipm_region_0_val)"
# query_string_gadm = "state_0 == @state_0_val & state_1 == @state_1_val"

OSM raw lines

In [4]:
osm_lines_raw_modified.other_tags.head(1)
# osm_lines_raw_modified.query(query_string_gadm).head(10)

0    { "tags.operator:short": "BPA", "tags.operator...
Name: other_tags, dtype: object

In [18]:
osm_lines_clean_modified.query(query_string_ipm).head(30)
# osm_lines_clean_modified.query(query_string_gadm).head(10)

,line_id,tag_type,voltage,circuits,tag_frequency,bus0,bus1,length,underground,under_construction,dc,country,geometry,sub_0_coors,sub_1_coors,state_0,ipm_region_0,state_1,ipm_region_1
19714,163145507-1,line,345000,1.0,50.0,NaN,NaN,NaN,False,False,False,US,LINESTRING (-119.97471350000016 40.07469180000...,POINT (-119.97471350000016 40.07469180000006),POINT (-120.63418130000011 41.512070900000104),US-NV,WECC_NNV,US-CA,WECC_PNW
69984,493763460-1,line,500000,1.0,0.0,NaN,NaN,NaN,False,False,True,US,LINESTRING (-119.96369430000028 41.99964609999...,POINT (-119.96369430000028 41.99964609999985),POINT (-119.20863230000018 39.94110770000004),US-OR,WECC_PNW,US-NV,WECC_NNV
90388,710704330-1,line,115000,1.0,50.0,NaN,NaN,NaN,False,False,False,US,LINESTRING (-117.84819469999991 41.69323289999...,POINT (-117.84819469999991 41.69323289999993),POINT (-117.7986862 41.99928440000003),US-NV,WECC_NNV,US-OR,WECC_PNW


EIA base network

In [19]:
eia_base_network_modified.query(query_string_ipm).head(20)
# eia_base_network_modified.query(query_string_gadm).head(10)

,OBJECTID_1,OBJECTID,ID,TYPE,STATUS,NAICS_CODE,NAICS_DESC,SOURCE,SOURCEDATE,VAL_METHOD,VAL_DATE,OWNER,v_nom,v_nom_class,INFERRED,SUB_1,SUB_2,SHAPE__Len,GlobalID,geometry,geometry_type,sub_0_coors,sub_1_coors,state_0,ipm_region_0,state_1,ipm_region_1
59786,73291,73291,204658,AC; Overhead,Not Available,221121,Electric Bulk Power Transmission and Control,Company Map,"Sat, 29 Aug 2015 00:00:00 GMT",Imagery/other,"Sat, 29 Aug 2015 00:00:00 GMT","Harney Electric Coop, Inc",115.0,100-161,No,Unknown202789,Unknown205318,48100.277288,b9cddb3a-0b0c-4422-b128-f0e843da886d,LINESTRING (-117.798710625187 41.9993558079948...,LineString,POINT (-117.798710625187 41.9993558079948),POINT (-117.84819762482 41.6932366338915),US-OR,WECC_PNW,US-NV,WECC_NNV
70836,84595,84595,304321,AC; Overhead,In Service,221121,Electric Bulk Power Transmission and Control,Western Electricity Coordinating Council https...,"Sun, 01 Jan 2017 00:00:00 GMT",Imagery/other,"Sun, 01 Jan 2017 00:00:00 GMT",Not Available,345.0,345,No,Not Available,Not Available,248829.835338,1b61313a-a316-4390-b232-95107e3647b8,LINESTRING (-119.974143007911 40.0748319362325...,LineString,POINT (-119.974143007911 40.0748319362325),POINT (-120.634433707017 41.5129921232001),US-NV,WECC_NNV,US-CA,WECC_PNW


In [38]:
base_network_pypsa_earth_modified.lines.query(query_string_ipm)
# base_network_pypsa_earth_modified.lines.query(query_string_gadm).head(10)

,Line,tag_type,v_nom,num_parallel,tag_frequency,bus0,bus1,length,underground,under_construction,dc,country,geometry,bounds,bus_0_coors,bus_1_coors,bus0_lon,bus0_lat,bus1_lon,bus1_lat,carrier,type,s_max_pu,s_nom,underwater_fraction,state_0,ipm_region_0,state_1,ipm_region_1,v_nom_class,x,r,g,b,s_nom_extendable,s_nom_min,s_nom_max,capital_cost,build_year,lifetime,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1745,1745,line,120.0,1.0,50.0,3143,585,5.329181,False,False,False,US,"LINESTRING (-120.0077 39.5222, -119.9079519000...","MULTIPOINT ((-120.0077 39.5222), (-119.7508 39...",POINT (-120.0077 39.5222),POINT (-119.7508 39.4878),-120.0077,39.5222,-119.7508,39.4878,AC,243-AL1/39-ST1A 20.0,0.7,134.060733,0.0,US-CA,WEC_CALN,US-NV,WECC_NNV,100-161,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8310,8315,line,345.0,1.0,50.0,10791,587,30.337267,False,False,False,US,"LINESTRING (-120.0042 39.6561, -120.0039273000...","MULTIPOINT ((-120.0042 39.6561), (-119.7488 39...",POINT (-120.0042 39.6561),POINT (-119.7488 39.4898),-120.0042,39.6561,-119.7488,39.4898,AC,Al/St 240/40 4-bundle 380.0,0.7,1541.698424,0.0,US-CA,WEC_CALN,US-NV,WECC_NNV,345,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8311,8316,line,345.0,1.0,50.0,10791,10792,68.724821,False,False,False,US,"LINESTRING (-120.0042 39.6561, -120.0044449000...","MULTIPOINT ((-120.0042 39.6561), (-119.9746 40...",POINT (-120.0042 39.6561),POINT (-119.9746 40.0754),-120.0042,39.6561,-119.9746,40.0754,AC,Al/St 240/40 4-bundle 380.0,0.7,1541.698424,0.0,US-CA,WEC_CALN,US-NV,WECC_NNV,345,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
base_network_pypsa_earth_modified.lines.query("v_nom == 765").head(1)

,Line,tag_type,v_nom,num_parallel,tag_frequency,bus0,bus1,length,underground,under_construction,dc,country,geometry,bounds,bus_0_coors,bus_1_coors,bus0_lon,bus0_lat,bus1_lon,bus1_lat,carrier,type,s_max_pu,s_nom,underwater_fraction,state_0,ipm_region_0,state_1,ipm_region_1,v_nom_class,x,r,g,b,s_nom_extendable,s_nom_min,s_nom_max,capital_cost,build_year,lifetime,terrain_factor,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2124,2124,line,765.0,1.0,60.0,1752,2336,255.945473,False,False,False,US,"LINESTRING (-81.7066 36.911, -81.6793665999997...","MULTIPOINT ((-81.7066 36.911), (-82.6103 38.181))",POINT (-81.7066 36.911),POINT (-82.6103 38.181),-81.7066,36.911,-82.6103,38.181,AC,Al/St 560/50 4-bundle 750.0,0.7,5512.07849,0.0,US-VA,PJM_West,US-KY,PJM_West,735 And Above,0.0,0.0,0.0,0.0,False,0.0,inf,0.0,0,inf,1.0,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
osm_lines_raw_path = pathlib.Path(
    "osm_raw_{}_to_{}.csv".format(ipm_region_0_val, ipm_region_1_val)
)
osm_lines_clean_path = pathlib.Path(
    "osm_clean_{}_to_{}.csv".format(ipm_region_0_val, ipm_region_1_val)
)
osm_raw_output_df = osm_lines_raw_modified.query(query_string_ipm)
osm_raw_output_df.to_csv(osm_lines_raw_path)
osm_clean_output_df = osm_lines_clean_modified.query(query_string_ipm)
osm_clean_output_df.to_csv(osm_lines_clean_path)

In [21]:
def plot_interactive(
    pypsa_df,
    eia_df,
    ipm_region_val_0,
    ipm_region_val_1,
    column_interactive_map,
    pypsa_html,
    eia_html,
):
    from matplotlib.colors import ListedColormap

    mycolor = ListedColormap("black")

    string_query_ipm = "(ipm_region_0 == @ipm_region_val_0 & ipm_region_1 == @ipm_region_val_1) | (ipm_region_0 == @ipm_region_val_1 & ipm_region_1 == @ipm_region_val_0)"

    eia_df["OBJECTID"] = eia_df["OBJECTID"].astype(str)

    pypsa_gdf = gpd.GeoDataFrame(
        pypsa_df, geometry=gpd.GeoSeries.from_wkt(pypsa_df.geometry), crs="EPSG:4326"
    )
    eia_gdf = gpd.GeoDataFrame(
        eia_df, geometry=gpd.GeoSeries.from_wkt(eia_df.geometry), crs="EPSG:4326"
    )

    pypsa_gdf_ipm = pypsa_gdf.query(string_query_ipm)
    eia_gdf_ipm = eia_gdf.query(string_query_ipm)

    print("pypsa dataframe shape", pypsa_gdf_ipm.shape)
    print("eia dataframe shape", eia_gdf_ipm.shape)

    eia_map = eia_gdf_ipm.explore(column="OBJECTID", cmap=mycolor, legend=False)
    eia_map.save(eia_html)

    pypsa_map = pypsa_gdf_ipm.explore(
        column=column_interactive_map, cmap=mycolor, legend=False
    )
    pypsa_map.save(pypsa_html)

In [22]:
pypsa_html_path = pathlib.Path(
    "html_map", "osm_clean_{}_to_{}.html".format(ipm_region_0_val, ipm_region_1_val)
)
eia_html_path = pathlib.Path(
    "html_map", "eia_{}_to_{}.html".format(ipm_region_0_val, ipm_region_1_val)
)
plot_interactive(
    osm_lines_clean_modified,
    eia_base_network_modified,
    ipm_region_0_val,
    ipm_region_1_val,
    "line_id",
    pypsa_html_path,
    eia_html_path,
)

pypsa dataframe shape (3, 19)
eia dataframe shape (2, 27)


In [ ]:
## ipm_region_capacities.loc[ipm_region_capacities["IPM"] < ipm_region_capacities["PyPSA"]].info()

In [23]:
osm_lines_raw_modified.head(1)

,Unnamed: 0,id,Type,Region,tags.cables,tags.name,tags.operator,tags.operator:wikidata,tags.power,tags.voltage,tags.circuits,tags.frequency,tags.source,tags.wires,tags.line,other_tags,geometry,sub_0_coors,sub_1_coors,state_0,ipm_region_0,state_1,ipm_region_1
0,0,5039311,way,US,3,Bonneville PH 1-Hood River No 1,Bonneville Power Administration,Q2910288,line,115000,NaN,NaN,NaN,NaN,NaN,"{'tags.operator:short': 'BPA', 'tags.operator:...",LINESTRING (-121.9476898999998 45.639100500000...,POINT (-121.9476898999998 45.63910050000022),POINT (-121.524595 45.6880996),US-OR,WECC_PNW,US-OR,WECC_PNW


In [6]:
ipm_gdp_usa_level_1["ipm_region_centroid"] = ipm_gdp_usa_level_1.centroid
ipm_gdp_usa_level_1.head(2)

,IPM_Region,geometry,ipm_region_centroid
0,ERC_PHDL,"MULTIPOLYGON (((1743524.086 -289710.658, 17444...",POINT (1672047.206 -210145.228)
1,ERC_REST,"MULTIPOLYGON (((2296018.210 -1049841.327, 2296...",POINT (2145446.940 -597062.788)


In [8]:
ipm_region_capacities_earth["ipm_region_0_centroid"] = ipm_region_capacities_earth[
    "ipm_region_0"
].map(dict(ipm_gdp_usa_level_1[["IPM_Region", "ipm_region_centroid"]].values))
ipm_region_capacities["ipm_region_1_centroid"] = ipm_region_capacities[
    "ipm_region_1"
].map(dict(ipm_gdp_usa_level_1[["IPM_Region", "ipm_region_centroid"]].values))
ipm_region_capacities["geometry"] = ipm_region_capacities.apply(
    lambda x: spl.LineString([x.ipm_region_0_centroid, x.ipm_region_1_centroid]), axis=1
)
ipm_region_capacities = ipm_region_capacities.drop(
    ["ipm_region_0_centroid", "ipm_region_1_centroid"], axis=1
)
# ipm_region_capacities = ipm_region_capacities.query("factor_IPM_over_PyPSA <= 1.0")
# ipm_region_capacities["factor_IPM_over_PyPSA"] = ipm_region_capacities["factor_IPM_over_PyPSA"].clip(upper=1.0)
ipm_region_capacities = ipm_region_capacities.query("factor_IPM_over_PyPSA > 1.0")
ipm_geo_data = gpd.GeoDataFrame(
    ipm_region_capacities, geometry=ipm_region_capacities.geometry, crs="EPSG:3310"
)

In [9]:
ipm_map_line = ipm_geo_data.explore(
    column="factor_IPM_over_PyPSA", cmap="Reds_r", style_kwds={"weight": 5.0}
)
ipm_map_line.save("interactive_map_line_outliers.html")

In [3]:
ipm_region_capacities.query("factor_IPM_over_PyPSA > 1.0")

,ipm_region_0,ipm_region_1,IPM,PyPSA,Error wrt IPM (%),Error wrt PyPSA (%),factor_IPM_over_PyPSA,coalesce
8,MIS_AR,SPP_N,167.0,77.084921,-53.841365,-116.644186,2.166442,MIS_AR-->SPP_N
33,MIS_MAPP,MIS_MNWI,5000.0,4213.975692,-15.720486,-18.652796,1.186528,MIS_MAPP-->MIS_MNWI
35,MIS_MIDA,MIS_MO,1574.0,179.864816,-88.572756,-775.101665,8.751017,MIS_MIDA-->MIS_MO
47,NENGREST,NY_Z_D,150.0,128.474869,-14.350088,-16.754352,1.167544,NENGREST-->NY_Z_D
50,NENG_CT,NY_Z_K,760.0,154.169842,-79.714494,-392.962818,4.929628,NENG_CT-->NY_Z_K
52,NY_Z_A,PJM_PENE,1000.0,642.374343,-35.762566,-55.672469,1.556725,NY_Z_A-->PJM_PENE
75,PJM_SMAC,PJM_WMAC,2500.0,2388.515384,-4.459385,-4.667528,1.046675,PJM_SMAC-->PJM_WMAC
80,SPP_N,SPP_SPS,2094.0,1670.173292,-20.240053,-25.376212,1.253762,SPP_N-->SPP_SPS
88,S_C_KY,S_C_TVA,1386.0,1053.493923,-23.990337,-31.562221,1.315622,S_C_KY-->S_C_TVA
89,S_C_TVA,S_D_AECI,600.0,179.864816,-70.022531,-233.583862,3.335839,S_C_TVA-->S_D_AECI


In [11]:
ipm_region_capacities_earth.describe()

,IPM,PyPSA,Error wrt IPM (%),Error wrt PyPSA (%),factor_IPM_over_PyPSA
count,122.000000,122.000000,122.000000,122.000000,122.000000
mean,2309.367082,19289.608516,908.360658,52.785839,0.472142
std,1919.227871,33933.200397,1626.150180,96.803329,0.968033
min,30.000000,77.084921,-88.572756,-775.101665,0.008585
25%,781.250000,2771.984938,109.559113,52.278761,0.094705
50%,2108.000000,6847.151913,305.294354,75.326416,0.246736
75%,3055.500000,19078.517995,955.923774,90.529548,0.477212
max,10555.000000,244282.115254,11548.388091,99.141512,8.751017


In [10]:
ipm_region_capacities_usa.describe()

,IPM,PyPSA,Error wrt IPM (%),Error wrt PyPSA (%),factor_IPM_over_PyPSA
count,121.000000,121.000000,121.000000,121.000000,121.000000
mean,2313.766810,14041.968182,1026.019056,49.741033,0.502590
std,1934.977543,15726.330936,1935.357262,138.979975,1.389800
min,30.000000,115.890000,-90.669663,-971.772640,0.008449
25%,775.000000,4557.500000,146.167242,59.377211,0.097051
50%,2122.000000,8753.890000,446.892680,81.714877,0.182851
75%,3074.000000,18873.560000,930.391365,90.294950,0.406228
max,10555.000000,94606.360000,11735.740000,99.155101,10.717726


In [42]:
ipm_region_capacities_usa_snom_above_5mw.describe()

,IPM,PyPSA,Error wrt IPM (%),Error wrt PyPSA (%),factor_IPM_over_PyPSA
count,121.000000,121.000000,121.000000,121.000000,121.000000
mean,2313.766810,14041.968099,1026.019053,49.741032,0.502590
std,1934.977543,15726.330978,1935.357264,138.979975,1.389800
min,30.000000,115.890000,-90.669663,-971.772640,0.008449
25%,775.000000,4557.500000,146.167242,59.377211,0.097051
50%,2122.000000,8753.890000,446.892680,81.714877,0.182851
75%,3074.000000,18873.560000,930.391365,90.294950,0.406228
max,10555.000000,94606.360000,11735.740000,99.155101,10.717726


In [51]:
capacity_df.head(10)

,ipm_region_0,ipm_region_1,IPM,PyPSA_Earth,PyPSA_USA,USA/Earth,geometry
0,ERC_PHDL,ERC_WEST,3702.0,27056.807338,2768.00,9.774858,LINESTRING (-11314030.372883672 4118896.638051...
1,ERC_REST,ERC_WEST,10555.0,101495.146235,22830.00,4.445692,LINESTRING (-10859885.476618178 3556127.007223...
2,ERC_REST,SPP_WEST,600.0,41934.197128,8782.22,4.774897,LINESTRING (-10859885.476618178 3556127.007223...
3,FRCC,S_SOU,3600.0,7629.172852,24119.59,0.316306,LINESTRING (-9119809.319695637 3295415.1009754...
4,MIS_AMSO,MIS_D_MS,1200.0,6036.084481,5412.01,1.115313,LINESTRING (-10054849.04658418 3506151.9355422...
5,MIS_AMSO,MIS_LA,2402.0,82655.144626,9343.16,8.846594,LINESTRING (-10054849.04658418 3506151.9355422...
6,MIS_AR,MIS_LA,2688.0,5368.015164,4656.26,1.152860,LINESTRING (-10208303.073294733 4143580.341891...
7,MIS_AR,SPP_N,167.0,77.084921,125.89,0.612320,LINESTRING (-10208303.073294733 4143580.341891...
8,MIS_AR,SPP_WEST,1571.0,13614.984559,28406.66,0.479288,LINESTRING (-10208303.073294733 4143580.341891...
9,MIS_AR,S_C_TVA,2546.0,5008.285532,6050.11,0.827801,LINESTRING (-10208303.073294733 4143580.341891...


In [17]:
merged_df = pd.merge(
    ipm_region_capacities_earth, ipm_region_capacities_usa, how="inner", on="coalesce"
)
merged_df["USA_over_Earth"] = merged_df["PyPSA_x"] / merged_df["PyPSA_y"]
capacity_df = merged_df.loc[
    :,
    (
        "ipm_region_0_x",
        "ipm_region_1_x",
        "IPM_x",
        "PyPSA_x",
        "PyPSA_y",
        "USA_over_Earth",
    ),
]
capacity_df = capacity_df.rename(
    columns={
        "ipm_region_0_x": "ipm_region_0",
        "ipm_region_1_x": "ipm_region_1",
        "IPM_x": "IPM",
        "PyPSA_x": "PyPSA_Earth",
        "PyPSA_y": "PyPSA_USA",
    }
)
ipm_shapes_gdf = ipm_gdp_usa_level_1.to_crs("3857")
ipm_shapes_gdf["ipm_region_centroid"] = ipm_shapes_gdf.centroid
capacity_df["ipm_region_0_centroid"] = capacity_df["ipm_region_0"].map(
    dict(ipm_shapes_gdf[["IPM_Region", "ipm_region_centroid"]].values)
)
capacity_df["ipm_region_1_centroid"] = capacity_df["ipm_region_1"].map(
    dict(ipm_shapes_gdf[["IPM_Region", "ipm_region_centroid"]].values)
)
capacity_df["geometry"] = capacity_df.apply(
    lambda x: spl.LineString([x.ipm_region_0_centroid, x.ipm_region_1_centroid]), axis=1
)
capacity_df = capacity_df.drop(
    ["ipm_region_0_centroid", "ipm_region_1_centroid"], axis=1
)
capacity_df_normal = capacity_df.query("USA_over_Earth <= 3.0")
capacity_df_outlier = capacity_df.query("USA_over_Earth > 3.0")
ipm_geo_data_normal = gpd.GeoDataFrame(
    capacity_df_normal, geometry=capacity_df_normal.geometry, crs="EPSG:3857"
)
ipm_geo_data_outlier = gpd.GeoDataFrame(
    capacity_df_outlier, geometry=capacity_df_outlier.geometry, crs="EPSG:3857"
)
ipm_geo_data_normal = ipm_geo_data_normal.explore(
    column="USA_over_Earth", cmap="jet", style_kwds={"weight": 5.0}
)
ipm_geo_data_normal.save("USA_over_Earth_interactive_map_line_normal.html")
ipm_geo_data_outlier = ipm_geo_data_outlier.explore(
    column="USA_over_Earth", cmap="Reds_r", style_kwds={"weight": 5.0}
)
ipm_geo_data_outlier.save("USA_over_Earth_interactive_map_line_outlier.html")

In [2]:
reeds_shapes_path = pathlib.Path(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-usa/workflow/repo_data/Reeds_Shapes/rb_and_ba_areas.shp"
)

In [3]:
reeds_shapes_gdf = gpd.read_file(reeds_shapes_path)

In [4]:
reeds_map = reeds_shapes_gdf.explore()
reeds_map.save("reeds_model_zones.html")

In [8]:
reeds_shapes_gdf.head(1)

,rb,BA_Code,geometry
0,p1,BPAT,"MULTIPOLYGON (((-13687924.240 5965691.032, -13..."


In [17]:
df = pd.read_csv(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/geothermal-modelling/analysis/outputs/pypsa_earth_reeds_capacities_after_concat.csv"
)

In [18]:
df.head(1)

,reeds_0,reeds_1,Capacity (MW),source
0,NaN,p1,4725.640821,PyPSA


In [3]:
output_df = pd.read_csv(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/geothermal-modelling/analysis/outputs/pypsa_earth_reeds_capacities_max_import_export.csv"
)
output_df.head(1)

,index,reeds_0,reeds_1,PyPSA,reeds,Error wrt reeds (%),Error wrt PyPSA (%),factor_reeds_over_PyPSA
0,0,p1,p2,37715.752745,9392.247,301.562616,75.097283,0.249027


In [4]:
output_df.describe()

,index,PyPSA,reeds,Error wrt reeds (%),Error wrt PyPSA (%),factor_reeds_over_PyPSA
count,277.000000,277.000000,277.000000,277.000000,277.000000,277.000000
mean,160.101083,11105.379934,2685.327217,407.605878,37.317937,0.626821
std,94.787275,17206.670628,2815.368730,817.751766,128.609089,1.286091
min,0.000000,77.084921,13.751000,-94.299620,-1654.269161,0.013020
25%,77.000000,2004.207951,844.379000,46.086568,31.547437,0.205757
50%,164.000000,5008.285532,1811.380000,149.400715,59.903884,0.400961
75%,244.000000,11196.584803,3785.197000,386.009473,79.424269,0.684526
max,320.000000,126907.475254,23673.977000,7580.277366,98.697964,17.542692


In [28]:
buses = pd.read_csv(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-eur/data/osm-prebuilt/buses.csv"
)
buses_magic = pd.read_csv(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-eur/buses_magic.csv"
)
europe_shapes = gpd.read_file(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-eur/resources/europe_shape.geojson"
).loc[0, "geometry"]
europe_shape_prepped = spl.prepared.prep(europe_shapes)

In [29]:
from shapely.geometry import Point

buses_in_europe_b_magic = buses_magic[["x", "y"]].apply(
    lambda p: europe_shape_prepped.contains(Point(p)), axis=1
)
buses_in_europe_b = buses[["x", "y"]].apply(
    lambda p: europe_shape_prepped.contains(Point(p)), axis=1
)

In [35]:
buses.query("index == 5090 or index == 5091")

,bus_id,voltage,dc,symbol,under_construction,x,y,country,geometry
5090,5231,380.0,f,Substation,f,6.8884,45.6783,IT,POINT (6.8884 45.6783)
5091,5232,400.0,f,Substation,f,6.8894,45.6793,IT,POINT (6.8894 45.6793)


In [30]:
print(buses_in_europe_b_magic)

0    False
1    False
dtype: bool


In [36]:
df_buses_output = pd.read_csv(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-eur/df_buses_output.csv"
)

In [37]:
df_buses_reference = pd.read_csv(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-eur/df_buses_reference.csv"
)

In [39]:
df_buses_output

,bus_id,v_nom,symbol,under_construction,x,y,country,geometry,carrier
0,5231,380.0,Substation,False,6.8884,45.6783,IT,POINT (6.8884 45.6783),AC
1,5232,400.0,Substation,False,6.8894,45.6793,IT,POINT (6.8894 45.6793),AC


In [40]:
df_buses_reference

,bus_id,v_nom,symbol,under_construction,x,y,country,geometry,carrier
0,5231,380.0,Substation,False,6.8884,45.6783,IT,POINT (6.8884 45.6783),AC
1,5232,400.0,Substation,False,6.8894,45.6793,IT,POINT (6.8894 45.6793),AC


In [41]:
df_buses_output.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bus_id              2 non-null      int64  
 1   v_nom               2 non-null      float64
 2   symbol              2 non-null      object 
 3   under_construction  2 non-null      bool   
 4   x                   2 non-null      float64
 5   y                   2 non-null      float64
 6   country             2 non-null      object 
 7   geometry            2 non-null      object 
 8   carrier             2 non-null      object 
dtypes: bool(1), float64(3), int64(1), object(4)
memory usage: 258.0+ bytes


In [42]:
df_buses_reference.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bus_id              2 non-null      int64  
 1   v_nom               2 non-null      float64
 2   symbol              2 non-null      object 
 3   under_construction  2 non-null      bool   
 4   x                   2 non-null      float64
 5   y                   2 non-null      float64
 6   country             2 non-null      object 
 7   geometry            2 non-null      object 
 8   carrier             2 non-null      object 
dtypes: bool(1), float64(3), int64(1), object(4)
memory usage: 258.0+ bytes


In [97]:
buses = pd.read_csv(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-eur/buses_test.csv"
).set_index("bus_id")
converters_path = pathlib.Path(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-eur/converters_test.csv"
)
# converters_path = pathlib.Path("/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-eur/data/osm-prebuilt/converters.csv")
converters_first = pd.read_csv(
    converters_path,
    quotechar="'",
    true_values=["t"],
    false_values=["f"],
    dtype=dict(converter_id="str", bus0="str", bus1="str"),
).set_index("converter_id")
converters_first = converters_first.drop("Unnamed: 5", axis=1)
converters_first

,bus0,bus1,voltage,geometry
converter_id,,,,
convert_0_1,5231,5232,380.0,"LINESTRING(6.8884 45.6783 ,6.8894 45.6793)"


In [92]:
buses

,v_nom,symbol,under_construction,x,y,country,geometry,carrier
bus_id,,,,,,,,
5231,380.0,Substation,False,6.8884,45.6783,IT,POINT (6.8884 45.6783),AC
5232,380.0,Substation,False,6.8894,45.6793,IT,POINT (6.8894 45.6793),AC


In [93]:
buses.index

Index([5231, 5232], dtype='int64', name='bus_id')

In [80]:
converters = pd.DataFrame(
    converters_first.loc[
        converters_first.bus0.isin(buses.index)
        & converters_first.bus1.isin(buses.index)
    ]
)
converters

,bus0,bus1,voltage,geometry,Unnamed: 5
converter_id,,,,,


In [5]:
links_ref = pd.read_csv(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-eur/df_links_reference.csv"
)
links_out = pd.read_csv(
    "/Users/fabriziofinozzi/Desktop/OpenEnergyTransition/repo/pypsa-eur/links_output.csv"
)

In [6]:
links_ref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          1 non-null      int64  
 1   link_id             1 non-null      object 
 2   bus0                1 non-null      int64  
 3   bus1                1 non-null      int64  
 4   voltage             1 non-null      float64
 5   p_nom               1 non-null      float64
 6   length              1 non-null      float64
 7   underground         1 non-null      bool   
 8   under_construction  1 non-null      bool   
 9   geometry            1 non-null      object 
 10  carrier             1 non-null      object 
dtypes: bool(2), float64(3), int64(3), object(3)
memory usage: 202.0+ bytes


In [7]:
links_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           1 non-null      int64  
 1   link_id              1 non-null      object 
 2   bus0                 1 non-null      int64  
 3   bus1                 1 non-null      int64  
 4   voltage              1 non-null      float64
 5   p_nom                1 non-null      float64
 6   length               1 non-null      float64
 7   underground          1 non-null      bool   
 8   under_counstruction  1 non-null      bool   
 9   geometry             1 non-null      object 
 10  carrier              1 non-null      object 
dtypes: bool(2), float64(3), int64(3), object(3)
memory usage: 202.0+ bytes
